###Deep Learning Lab Manual CAI2503, Prepared By IC: Pakruddin.B, Asst.Prof., PSCSE,PU. 2025-26

### Lab 12: Transfer Learning with CNNs (VGG16)
# -------------------------------------------
Objective: Use pre-trained VGG16 for CIFAR-10 classification via transfer learning

Duration: ~2 hours

###What Students Will Learn
What transfer learning is and why it’s useful.

How to use VGG16 pre-trained on ImageNet for a new dataset.

How to freeze base layers and train only the classification head.

How preprocessing is critical for transfer learning.

###Expected Outputs
Accuracy plot showing fast convergence due to pre-trained weights.

Loss plot showing stable learning.

Sample predictions grid with predicted and true labels.

In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# 1. Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Resize CIFAR-10 images from 32x32 to 224x224 (VGG16 input size)
x_train_resized = tf.image.resize(x_train, (224, 224))
x_test_resized = tf.image.resize(x_test, (224, 224))

# Preprocess input according to VGG16's requirements
x_train_prep = preprocess_input(x_train_resized)
x_test_prep = preprocess_input(x_test_resized)

# 2. Load VGG16 without top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base layers

# 3. Build transfer learning model
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 4. Train model
history = model.fit(
    x_train_prep, y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=32,
    verbose=1
)

# 5. Evaluate on test set
test_loss, test_acc = model.evaluate(x_test_prep, y_test, verbose=0)
print(f"\nTest Accuracy: {test_acc:.4f}")

# 6. Plot training vs validation accuracy
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Transfer Learning - VGG16 Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

# 7. Plot training vs validation loss
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Transfer Learning - VGG16 Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

# 8. Sample predictions
preds = model.predict(x_test_prep[:10])
pred_labels = np.argmax(preds, axis=1)

class_names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer',
               'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(x_test[i])
    plt.title(f"Pred: {class_names[pred_labels[i]]}\nTrue: {class_names[int(y_test[i])]}")
    plt.axis('off')
plt.tight_layout()
plt.show()



# 9. Generate predictions for the entire test set
y_pred_probs = model.predict(x_test_prep)
y_pred = np.argmax(y_pred_probs, axis=1)

# Flatten true labels
y_true = y_test.flatten()

# 10. Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix - VGG16 Transfer Learning CIFAR-10")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# 11. Classification Report
print("\nClassification Report - VGG16 Transfer Learning CIFAR-10")
print(classification_report(y_true, y_pred, target_names=class_names))


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
